In [0]:
! pip install --upgrade git+https://github.com/e-lo/forecastcards#egg=forecastcards
try:
  import forecastcards
except:
  ! pip install --upgrade git+https://github.com/e-lo/forecastcards#egg=forecastcards
  import forecastcards

# Estimate Quantile Regression Models

## Create estimation dataset
- Finds
- Combines
- Cleans
- Adds categorical variables
- Scales

In [0]:
estimate_df  = forecastcards.default_data_clean(forecastcards.combine_data(forecastcards.map_cards()))

In [0]:
estimate_df.describe()
estimate_df

## Models 

- Try a linear model first
- Estimate quantiles and plot

In [0]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.quantile_regression import QuantReg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#### Linear Model

In [0]:
## Define variables
X = estimate_df["obs_value"]
y = estimate_df["forecast_value"]
X = sm.add_constant(X)

# Fit
model = sm.OLS(y, X).fit() 
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

#### Quantile Regression

In [0]:
quantiles = np.arange(.1, .90, .2)

def fit_model(q):
    res = mod.fit(q=q)
    return [q, res.params['Intercept'], res.params['forecast_value']] + \
            res.conf_int().loc['forecast_value'].tolist()
    
models = [fit_model(x) for x in quantiles]
models = pd.DataFrame(models, columns=['q', 'a', 'b','lb','ub'])

ols = smf.ols('obs_value ~ forecast_value', estimate_df).fit()
ols_ci = ols.conf_int().loc['forecast_value'].tolist()
ols = dict(a = ols.params['Intercept'],
           b = ols.params['forecast_value'],
           lb = ols_ci[0],
           ub = ols_ci[1])

#print(models)
#print(ols)

In [0]:
x = np.arange(estimate_df['forecast_value'].min(), estimate_df['forecast_value'].max(), 50)
get_y = lambda a, b: a + b * x

fig, ax = plt.subplots(figsize=(8, 6))

for i in range(models.shape[0]):
    y = get_y(models.a[i], models.b[i])
    ax.plot(x, y, linestyle='dotted', color='grey')
    
y = get_y(ols['a'], ols['b'])

ax.plot(x, y, color='red', label='OLS')
ax.scatter(estimate_df['forecast_value'], estimate_df['obs_value'], alpha=.2)
#ax.set_xlim((240, 3000))
#ax.set_ylim((240, 2000))
legend = ax.legend()
ax.set_xlabel('Forecast Value', fontsize=16)
ax.set_ylabel('Observed Value', fontsize=16);